# Basic useage of the DSMS-Python-SDK

Before you run this tutorial: make sure to have access to an DSMS-instance of your interest, that you have installed this package and that you have copied the needed variables such as the `DSMS_HOST_URL` and `DSMS_TOKEN` into an `.env`-file.

# pending

Also make sure to pip install dsms-sdk and how to export DSMS_HOST_URL and DSMS_TOKEN, attach.

First of all, make let us import the needed classes and functions for this tutortial.

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [ ]:
#specify path to an arbitrary file
env = os.path.join("..", ".env")

# start the session
load_dotenv(env)

dsms = DSMS()

: 

### 1: Introduction

We can see which kind of DSMS-object we own as a user:

In [ ]:
dsms.kitems

We can investigate what a KItem needs in order to be created. KItems are entirely based on [`Pydantic`](https://docs.pydantic.dev/latest/)-Models (v2), hence the properties (in `Pydantic` called `Fields`) are automatically validated once we set them. 

The schema of the KItem itself is a JSON schema which is machine-readable and can be directly incorporated into [Swagger](https://swagger.io/tools/swagger-ui/)-supported APIs like e.g. [`FastAPI`](https://fastapi.tiangolo.com/).

In [ ]:
pprint(KItem.model_json_schema())

We can investigate the KTypes defined in the remote instance:

In [ ]:
for ktype in dsms.ktypes:
    print(ktype)

### 2: Create KItems

We can make new KItems by simple class-initiation:

In [ ]:
item = KItem(
    name="foo12345",
    ktype_id=dsms.ktypes.DatasetCatalog,
    custom_properties={"foo": "bar"},
)

item

Remember: changes are only syncronized with the DSMS when you call the `commit`-method:

In [ ]:
dsms.commit()

As we can see, the object we created before running the `commit`-method has automatically been updated, e.g. with the creation- and update-timestamp:

In [ ]:
item

And the list of our KItems in the DSMS has been updated as well:

In [ ]:
dsms.kitems

### 3. Update KItems

Now, we would like to update the properties of our KItem we created previously.

Depending on the schema of each property (see `KItem.model_schema_json()` in the **Introduction** of this tutorial), we can simply use the standard `list`-method as we know them from basic Python (e.g. for the `annotations`, `attachments`, `external_link`, etc). 


Other properties which are not `list`-like can be simply set by attribute-assignment (e.g. `name`, `slug`, `ktype_id`, etc).

In [ ]:
# specify the path to any arbitrary file to be uploaded
file = os.path.join("..", "README.md")

item.name = "foobar"
item.custom_properties.update({"foobar": "foobar"})
item.attachments.append({"name": file})
item.annotations.append(
    {
        "iri": "www.example.org/foo",
        "name": "example class",
        "namespace": "www.example.org",
    }
)
item.external_links.append(
    {"url": "http://example.org", "label": "example link"}
)
item.contacts.append({"name": "foo", "email": "foo@bar.mail"})
item.affiliations.append({"name": "foobar team"})
item.user_groups.append({"name": "foogroup", "group_id": "123"})

Changes are sent to the DSMS through the `commit`-method again.

In [ ]:
dsms.commit()

We can see now that e.g. the local system path of the attachment is changed to a simply file name, which means that the upload was successful. If not so, an error would have beem thrown during the `commit`.

Furthermore we can also download the file we uploaded again:

In [ ]:
for file in item.attachments:
    download = file.download()

    print("\t\t\t Downloaded file:", file)
    print("|------------------------------------Beginning of file------------------------------------|")
    print(download)
    print("|---------------------------------------End of file---------------------------------------|")

### 4: Delete KItems and their properties

We can also remove properties from the KItem without deleting the KItem itself.

For the `list`-like properties, we can use the standard `list`-methods from basic Python again (e.g. `pop`, `remove`, etc. or the `del`-operator).

For the other, non-`list`-like properties, we can simply use the attribute-assignment again.

When we only want single parts of the properties in the KItem, we can do it like this:

In [ ]:
item.attachments.pop(0)
item.annotations.pop(0)
item.external_links.pop(0)
item.contacts.pop(0)
item.user_groups.pop(0)

However, we can also reset the entire property by setting it to e.g. an empty list again:

In [ ]:
item.affiliations = []

See the changes:

In [ ]:
item

Send the changes to the DSMS with the `commit`-method:

In [ ]:
dsms.commit()

However, we can also delete the whole KItem from the DSMS by applying the `del`-operator to the `dsms`-object with the individual `KItem`-object:

In [ ]:
del dsms[item]

Commit the changes:

In [ ]:
dsms.commit()

As we can see know, our KItem has been removed from our inventory:

In [ ]:
dsms.kitems

### 5: Search for KItems

In the last unit of this tutorial, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also wnat to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`-attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [ ]:
item = KItem(
    name="foo 1",
    ktype_id=dsms.ktypes.DatasetCatalog
)

item2 = KItem(
    name="foo 2",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item],
    annotations=[
        {
            "iri": "www.example.org/foo",
            "name": "foo",
            "namespace": "www.example.org",
        }
    ],
)
item3 = KItem(
    name="foo 3", 
    ktype_id=dsms.ktypes.Organization
)
item4 = KItem(
    name="foo 4",
    ktype_id=dsms.ktypes.Organization,
    annotations=[
        {
            "iri": "www.example.org/bar",
            "name": "bar",
            "namespace": "https://www.example.org",
        }
    ],
)

dsms.commit()

Now, we are apply to search for e.g. kitems of type `DatasetCatalog`:

In [ ]:
dsms.search(ktypes=[dsms.ktypes.DatasetCatalog])

... and for all of type `Organization` and `DatasetCatalog`:

In [ ]:
dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.DatasetCatalog])

... or for all of type `DatasetCatalog` with `foo` in the name:

In [ ]:
dsms.search(query="foo", ktypes=[dsms.ktypes.DatasetCatalog])

... and for all of type `Organization` with the annotation `www.example.org/foo`:

In [ ]:
dsms.search(
        ktypes=[dsms.ktypes.Organization], annotations=["www.example.org/foo"]
    )

Clean up the DSMS from the tutortial:

In [ ]:
del dsms[item]
del dsms[item2]
del dsms[item3]
del dsms[item4]

dsms.commit()

dsms.kitems

### 6. Apps

We can investigate which apps are available through JupyterLab:

In [ ]:
dsms.apps

7. HDF5 

We are also able to upload dataframes or time series data and investigate them:

In [ ]:
data = {"a": list(range(100)), "b": list(range(1,101))}


item = KItem(name="testdata123", ktype_id=dsms.ktypes.DatasetCatalog, hdf5=data)
dsms.commit()

print("Column-wise:")
for column in item.hdf5:
    print("column:", column.name, ",\n", "data:", column.get())

df = item.hdf5.to_df()
print("\nAs data frame:")
print(df)

new_df = df.drop(['a'], axis=1)
item.hdf5 = new_df

dsms.commit()

In [ ]:
del dsms[item]

In [ ]:
dsms.commit()